In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
# import seaborn as sns
# sns.set()
# sns.set_style("white")
# sns.set_palette("GnBu_d")

# also import these "new" libraries 
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')

import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
from pathlib import Path

import geopy
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic



https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10

In [2]:
import math
from ast import literal_eval
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
top_left = '(42.41568810349132, -71.15337356574248)'
bottom_right = '(42.308814794642046, -70.9677292096557)'

In [4]:
merc(top_left)

(-7920757.313562025, 5223452.331299398)

In [5]:
merc(bottom_right)

(-7900091.4783737995, 5207351.240481618)

In [6]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON_RETINA, get_provider

tile_provider = get_provider(CARTODBPOSITRON_RETINA)

p = figure(x_range=(-7920757.313562025, -7900091.4783737995), y_range=(5223452.331299398, 5207351.240481618),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

output_notebook()

show(p)

Loading BokehJS ...

In [7]:
df_stations= pd.read_csv("../all_station_id_with_amounts.csv")

In [8]:
df_stations['coordinates'] = list(zip(df_stations["start_station_latitude"],df_stations["start_station_longitude"]))

In [9]:
df_stations.head()

,Unnamed: 0,Unnamed: 0.1,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station,coordinates
0,0,4748,1,18 Dorrance Warehouse,42.387151,-71.075978,78,"(42.387151, -71.075978)"
1,1,26681,3,Colleges of the Fenway,42.340021,-71.100812,4690,"(42.340021, -71.100812)"
2,2,97049,4,Tremont St. at Berkeley St.,42.345392,-71.069616,9005,"(42.345392, -71.069616)"
3,3,161457,5,Northeastern U / North Parking Lot,42.341814,-71.090179,7090,"(42.341814, -71.090179)"
4,4,103358,6,Cambridge St. at Joy St.,42.361174,-71.065142,11713,"(42.361174, -71.065142)"


In [10]:
def makestr(x):
    return (str(x))

In [11]:
df_stations['str_coords'] = df_stations['coordinates'].apply(lambda x: makestr(x))

In [12]:
df_stations['coords_x'] = df_stations['str_coords'].apply(lambda x: merc(x)[0])
df_stations['coords_y'] = df_stations['str_coords'].apply(lambda x: merc(x)[1])

In [13]:
df_stations.head(1)

,Unnamed: 0,Unnamed: 0.1,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station,coordinates,str_coords,coords_x,coords_y
0,0,4748,1,18 Dorrance Warehouse,42.387151,-71.075978,78,"(42.387151, -71.075978)","(42.387151, -71.075978)",-7.912142e+06,5.219150e+06


In [14]:

p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)
p.circle(x = df_stations['coords_x'],
         y = df_stations['coords_y'])
output_notebook()
show(p)

Loading BokehJS ...

In [15]:
df_stations.drop(['Unnamed: 0.1','Unnamed: 0','start_station_id','start_station_name','start_station_latitude','start_station_longitude','coordinates','str_coords'], axis=1, inplace=True)

In [16]:
df_stations.head(1)

,amout_of_trips_per_station,coords_x,coords_y
0,78,-7.912142e+06,5.219150e+06


In [17]:
Merged = df_stations

In [18]:
Merged['circle_sizes'] = Merged['amout_of_trips_per_station'] / 900

In [19]:
# range bounds supplied in web mercator coordinates
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)
p.circle(x=Merged['coords_x'],
         y=Merged['coords_y'], 
         size=Merged['circle_sizes'],
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05)
    
output_notebook()
show(p)

Loading BokehJS ...